In [45]:
import os
from dotenv import load_dotenv
from langchain.chains import RetrievalQA
from langchain_google_genai import GoogleGenerativeAIEmbeddings, ChatGoogleGenerativeAI
from langchain_community.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma

In [46]:
load_dotenv()
os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY")

In [47]:
#Load documents
loader = WebBaseLoader(["https://excellencetechnologies.in/"])
docs = loader.load()

In [48]:
docs

[Document(metadata={'source': 'https://excellencetechnologies.in/', 'title': 'Best Mobile And Web Application Development Company | Ecommerce Website Development Solutions | Excellence Technologies', 'description': 'Excellence Technologies is one of the Best E-Commerce websites and Mobile App development companies across the world focusing on iPhone & Android applications. We also offer Custom Website Design and Development services for web applications with Software Solutions across the globe.', 'language': 'en'}, page_content="\n\n\n\n\n\n\n\nBest Mobile And Web Application Development Company | Ecommerce Website Development Solutions | Excellence Technologies\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n   \n\n\n\n\n\n\n\n\n\n\nHome\n\n\n\n\nAbout\n\n\n\n\nServices\n\n\n\n\nCareer\n\n\n\n\nPortfolio\n\n\n\n\nBlog\n\n\n\n\nContact\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n        Your browser does not support HTML5 video.\n    \n\n\n        PROVIDING MODERNISTIC SOFTWARE S

In [49]:
# Splitting documents
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
splits = splitter.split_documents(docs)


In [50]:
splits

[Document(metadata={'source': 'https://excellencetechnologies.in/', 'title': 'Best Mobile And Web Application Development Company | Ecommerce Website Development Solutions | Excellence Technologies', 'description': 'Excellence Technologies is one of the Best E-Commerce websites and Mobile App development companies across the world focusing on iPhone & Android applications. We also offer Custom Website Design and Development services for web applications with Software Solutions across the globe.', 'language': 'en'}, page_content="Best Mobile And Web Application Development Company | Ecommerce Website Development Solutions | Excellence Technologies\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n   \n\n\n\n\n\n\n\n\n\n\nHome\n\n\n\n\nAbout\n\n\n\n\nServices\n\n\n\n\nCareer\n\n\n\n\nPortfolio\n\n\n\n\nBlog\n\n\n\n\nContact\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n        Your browser does not support HTML5 video.\n    \n\n\n        PROVIDING MODERNISTIC SOFTWARE SOLUTIONS FOR YOU

In [51]:
len(splits)

17

In [52]:
embedding = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
vectorstore = Chroma.from_documents(documents=splits, embedding=embedding)
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", temperature=0.9, max_output_tokens=100)

In [53]:
embedding

GoogleGenerativeAIEmbeddings(client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x000001D37FBD12E0>, model='models/embedding-001', task_type=None, google_api_key=SecretStr('**********'), credentials=None, client_options=None, transport=None, request_options=None)

In [54]:
vectorstore

In [55]:
llm

ChatGoogleGenerativeAI(model='models/gemini-2.0-flash', google_api_key=SecretStr('**********'), temperature=0.9, max_output_tokens=100, client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x000001D37FBD13D0>, default_metadata=(), model_kwargs={})

In [56]:
que_ans_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=vectorstore.as_retriever(),
    return_source_documents=True,
    verbose=True
)

In [57]:
chat_history = []
print("Ask anything about the website you have provided (type 'exit' to quit):")

while True:
    user_input = input("\nYou: ").strip()
    if user_input.lower() in ["exit", "quit"]:
        break

    relevant_docs = vectorstore.similarity_search(user_input, k=5)

    if not relevant_docs:
        print("AI: Sorry, I couldn't find anything related to that in the website content.")
        continue

    try:
        result = que_ans_chain.invoke({"query": user_input})
        ai_response = result['result']
        chat_history.append((user_input, ai_response))

        # Print chat history
        print("\n Conversation:")
        for idx, (q, a) in enumerate(chat_history, 1):
            print(f"\nYou {idx}: {q}")
            print(f"AI {idx}: {a}")

    except Exception as e:
        print("Error:", e)

Ask anything about the website you have provided (type 'exit' to quit):


> Entering new RetrievalQA chain...

> Finished chain.

 Conversation:

You 1: About the document
AI 1: This document appears to be about Excellence Technologies, a company that provides "modernistic software solutions". They are a mobile and web application development company that also offers e-commerce website development solutions. The document includes sections for "Home", "About", "Services", "Career", "Portfolio", "Blog", and "Contact". It also contains a video and a section with client testimonials.


> Entering new RetrievalQA chain...

> Finished chain.

 Conversation:

You 1: About the document
AI 1: This document appears to be about Excellence Technologies, a company that provides "modernistic software solutions". They are a mobile and web application development company that also offers e-commerce website development solutions. The document includes sections for "Home", "About", "Services", "Career",